In [ ]:
import pandas as pd
import numpy as np
import turbodbc
import _main as pp
import os
import re

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid")

from sklearn.preprocessing import Normalizer, StandardScaler, MinMaxScaler
from sklearn.cluster import KMeans

In [ ]:
def to_pivot(df):
    pvt = pd.DataFrame()
    cols = ['Preis_Konkurrenz',
            'Txt_Kurz_Konkurrenz',
            'Txt_Lang_Konkurrenz',
            'Joined_on',
            'Preisdifferenz',
            'Closest', 'Distance']
    for i in cols:
        pvt_ = df.pivot(index='UID', columns='Konkurrenz', values=i)
        pvt_.columns = [i+'_'+j for j in pvt_.columns]
        pvt = pd.concat([pvt, pvt_], axis=1)
    pvt = df.merge(
        pvt, left_on='UID',
        right_index=True)
    pvt.drop(cols, axis=1, inplace=True)
    pvt.drop('Konkurrenz', axis=1, inplace=True)
    pvt.drop_duplicates(inplace=True)
    return pvt.reset_index(drop=True)

In [ ]:
def get_companies(df):
    companies = [i[17:] for i in df.columns if i.startswith(
        "Preis_") and not i.endswith("Log")]
    return companies

In [ ]:
def prep_df(df):
    companies = get_companies(df)
    df.replace('', np.nan, inplace=True)
    df.replace('\n', '', inplace=True)
    df.replace('\r', '', inplace=True)
    for j, i in enumerate(companies):
        print("{} Company in data: {}".format(j, i))
    df['Discount_perc'] = (df['GrossSales_LTM'] - df['Sales_LTM']) / df['GrossSales_LTM']
    df['Margin_perc'] = df['Margin_LTM'] / df['Sales_LTM']

    for i in ['GrossSales_LTM', 'Sales_LTM',
              'Margin_LTM', 'Quantity_LTM',
              'ObjectRate', 'CountOfOrders', 'Preis',
              'CountOfCustomers', 'Discount_perc', 'Margin_perc']:
        df[i] = df[i].astype(np.float)
        df[i+'_Log'] = df[i].apply(lambda x: np.log(x+1))
    
    df['GrossSales_Caluclated'] = df['Preis'] * df['Quantity_LTM']
    
    for i in companies:
        df['Preis_Konkurrenz_'+i] = df['Preis_Konkurrenz_'+i].astype(np.float)
        df['Preis_Konkurrenz_'+i+'_Log'] = df['Preis_Konkurrenz_'+i].apply(lambda x: np.log(x+1))
        df['Outlier_'+i] = np.nan
        df['GrossSales_Caluclated_'+i] = df['Preis_Konkurrenz_'+i] * df['Quantity_LTM']
    
    return df

In [ ]:
def cluster(df, X=None, tag='', clusters=5, scaler_obj=StandardScaler):
    if tag in df.columns:
        df[tag] = np.nan
    if not X:
        X = df
    else:
        X = df[X]
    norm = scaler_obj()
    normalized = norm.fit_transform(X)
    kmeans = KMeans(
        n_clusters=clusters, random_state=23).fit(normalized)
    df[tag] = kmeans.labels_
    return df

In [ ]:
def outlier_detection(df, threshold=1.5):
    companies = get_companies(df)
    for i in companies:
        df_temp = df[['Preis', 'Preis_Konkurrenz_'+i]].replace('',np.nan).dropna(how='any').copy()
        df_temp['Preis'] = df_temp['Preis'].astype(np.float)
        df_temp['Preis_Konkurrenz_'+i] = df_temp['Preis_Konkurrenz_'+i].astype(np.float)
        df_temp['Diff'] = df_temp['Preis'] - df_temp['Preis_Konkurrenz_'+i]
        df['Outlier_'+i] = False
        df_temp['Outlier_'+i] = (np.abs(
                df_temp['Diff']) / df_temp['Preis']) > threshold * (
                np.std(df_temp['Diff']) / df_temp['Preis'])
        df.update(df_temp)
    return df

In [ ]:
def get_cluster(df, plot=True):
    mapper1 = {0: "Low Margin, Low Sales",
               1: "Low Margin, High Sales",
               2: "High Margin, Low Sales",
               3: "High Margin, High Sales"}

    mapper2 = {0: "Low Quantity, Medium Sales",
               1: "Medium Quantity, High Sales",
               2: "Medium Quantity, Medium Sales",
               3: "High Quantity, High Sales",
               4: "Low Quantity, Low Sales"}

    mapper3 = {0: "Low ObjectRate, Medium Sales",
               1: "Low Sales, High ObjectRate",
               2: "High Sales",
               3: "Low Sales, Low ObjectRate"}
    
    for i in [['Margin_perc', 'Sales_LTM_Log', StandardScaler, 4, mapper1],
              ['Quantity_LTM_Log', 'Sales_LTM_Log', StandardScaler, 5, mapper2],
              ['ObjectRate', 'Sales_LTM_Log', StandardScaler, 4, mapper3]]:

        tag = 'Cluster_'+'_'.join(i[:2])

        dfs = df.query("Margin_perc > 0.0001 and Sales_LTM > 100 and Margin_perc < 0.8").copy()
        dfs = dfs.pipe(cluster, X=[i[0], i[1]], tag=tag, clusters=i[3], scaler_obj=i[2])
        dfs[tag] = dfs[tag].map(i[4])
        
        if plot:
            sns.lmplot(i[0], i[1],
                       data=dfs, hue=tag,
                       fit_reg=False, size=7,
                       aspect=1.6,
                       palette=sns.color_palette('colorblind'))

            plt.savefig(os.path.join(pp.Path, "Plots","PDF", tag+".pdf"))
            plt.savefig(os.path.join(pp.Path, "Plots","PNG", tag+".png"))

        df[tag] = np.nan
        df.update(dfs[tag])
    return df

In [ ]:
def plot_outlier(df):
    for i in get_companies(df):
        dfplot = df[(df['Joined_on_'+i] != 'Text_Similarity') & (pd.notnull(
            df['Preis_Konkurrenz_'+i]))]
        f, ax = plt.subplots(figsize=(20, 20))
        p1 = sns.regplot('Preis_Konkurrenz_'+i, 'Preis', data=dfplot, fit_reg=False)    
        for line in range(0,dfplot.shape[0]):
            if dfplot.Outlier_Sanitas.iloc[line]:
                p1.text(dfplot.Preis_Konkurrenz_Sanitas.iloc[line]+0.5,
                        dfplot.Preis.iloc[line], dfplot.Art_Txt_Kurz.iloc[line],
                        horizontalalignment='left', size='medium', color='black')
        plt.savefig(os.path.join(pp.Path, "Plots","PDF", "Outlier_{}.pdf".format(i)))
        plt.savefig(os.path.join(pp.Path, "Plots","PGN", "Outlier_{}.pgn".format(i)))

In [ ]:
query = """
    SELECT UniqueId
      ,GrossSales_LTM = sum(GrossSales)
      ,Sales_LTM = sum(Sales)
      ,Margin_LTM = sum(Margin)
      ,Quantity_LTM = sum(Quantity)
      ,ObjectRate = max(ObjectRate)
      ,CountOfOrders = sum(CountOfOrders)
      ,CountOfCustomers = sum(CountOfCustomers)
    FROM (
    SELECT UniqueId = idItemOld
      ,GrossSales = sum(GrossSales)
      ,Sales = sum(Sales)
      ,Margin = sum(Margin)
      ,Quantity = sum(Quantity)
      ,ObjectRate = avg(case when PricingLong IN ('Offerte', 'Baustelle', 'Aktionspreise') then 1.0 else 0.0 end)
      ,CountOfOrders = count(distinct OrderNo)
      ,CountOfCustomers = count(distinct idCustomer)
    FROM CRHBUSADWH01.InfoPool.FACT.V_Sales s
      inner join CRHBUSADWH01.infopool.dim.v_item i on i.iditem = s.iditem and i.idbusinesssection = s.idbusinesssection
      inner join [CRHBUSADWH01].[InfoPool].[DIM].[V_Pricing] p on p.idpricing = s.idpricing
    where itemgroupgrouphierarchyname_l1 = '05-Sanitär'
    and Date > dateadd(month, -12, getdate())
    and Sales > 0
    group by idItemOld

    UNION ALL

    SELECT UniqueId = substring(i.iditemorigin, 2, 500)
      ,GrossSales = sum(GrossSales)
      ,Sales = sum(Sales)
      ,Margin = sum(Margin)
      ,Quantity = sum(Quantity)
      ,ObjectRate = 0
      ,CountOfOrders = count(distinct OrderNo)
      ,CountOfCustomers = count(distinct idCustomer)
    FROM InfoPool_MOV.FACT.Sales s
      inner join InfoPool_MOV.DIM.v_item i on i.iditem = s.iditem
    where itemgroupgrouphierarchyname_l1 = '05-Sanitär'
    and Date > dateadd(month, -12, getdate())
    and Sales > 0
    group by substring(i.iditemorigin, 2, 500)
    ) x
    GROUP BY UniqueId
    HAVING sum(Sales)>0
"""

In [ ]:
con = pp.create_connection_string_turbo("CRHBUSADWH02", 'AnalystCM')

In [ ]:
meta = pp.sql_to_pandas(con, query)

In [ ]:
files = [i for i in os.listdir(os.path.join(pp.Path, "Matched")) if i.endswith(".tsv")]
files.sort(reverse=True)
file = files[0]
file

In [ ]:
df = (pd.read_csv(os.path.join(pp.Path, "Matched", file), sep="\t", dtype=str)
        .pipe(to_pivot)
        .merge(meta, how='left', left_on='UID', right_on='UniqueId')
        .pipe(prep_df)
        .drop_duplicates(inplace=False)
        .pipe(outlier_detection, threshold=5)
        .pipe(get_cluster, plot=True))

In [ ]:
df[~df['Art_Txt_Kurz'].str.contains(
    'procasa', case=False)].to_csv(
    os.path.join(pp.Path, "Analyse", "Output_Analysis.tsv"),
    sep="\t", encoding='utf-8', index=True)